<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB42_Single_vs_Multi_Agent_version_am%C3%A9lior%C3%A9e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB42 : Mono Agent vers Multi Agent Interactions

**Objectif:** Expérimenter comment plusieurs agents légers coordonnent (ou entrent en conflit) par rapport à un agent unique accomplissant la même tâche.

**Durée estimée:** 15–20 minutes

**Livrable:** Un notebook qui exécute la même tâche en modes (A) single-agent et (B) multi-agent et compare les résultats.

---

## Architecture

- **Blackboard** (mémoire partagée)
- **Rôles** spécialisés (Researcher, Planner, Critic)
- **Protocole** de messages structuré
- **Comparaison** single-agent vs multi-agent
---

## ⚠️ Améliorations de cette version

✅ **Support multi-rounds fonctionnel** : Le Researcher collecte des faits complémentaires à chaque round

✅ **Contexte enrichi** : Prise en compte de l'historique des plans et critiques

✅ **Logs détaillés** : Statistiques après chaque round

✅ **Incrémentalité** : Chaque round améliore le précédent au lieu de répéter

---

## Architecture

- **Blackboard** (mémoire partagée)
- **Rôles** spécialisés (Researcher, Planner, Critic)
- **Protocole** de messages structuré
- **Comparaison** single-agent vs multi-agent

## 1) Setup :

Installation des dépendances nécessaires.

In [1]:
!pip install -q openai python-dotenv

### Configuration de la clé API OpenAI

⚠️ **Important:** Remplacez `your_api_key_here` par votre vraie clé API OpenAI.

Dans Google Colab, vous pouvez aussi utiliser les **Secrets** :
1. Cliquez sur l'icône 🔑 dans la barre latérale gauche
2. Ajoutez un secret nommé `OPENAI_API_KEY`
3. Décommentez la méthode alternative ci-dessous

In [2]:
import os
from google.colab import userdata

# Récupérer la clé API depuis les secrets Colab
# Pour ajouter : cliquez sur 🔑 dans le panneau de gauche
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = openai_api_key
    print("✓ Clé API OpenAI chargée depuis les secrets Colab")
except:
    print("⚠ Secrets Colab non configurés. Veuillez ajouter OPENAI_API_KEY.")
    print("Instructions : Cliquez sur 🔑 dans le panneau gauche > Ajouter un nouveau secret")

✓ Clé API OpenAI chargée depuis les secrets Colab


## 2) Define a minimal agent interface :

Chaque agent possède :
- **role** : son rôle spécifique
- **prompt** : instructions système
- **act(state)** : méthode retournant un message et des mises à jour optionnelles du blackboard

In [3]:
import os
import uuid
import time
from dataclasses import dataclass, field
from typing import Dict, Any, List
from openai import OpenAI

# Initialisation du client OpenAI
llm = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

def chat(system: str, user: str, temp: float = 0) -> str:
    """
    Fonction helper pour appeler l'API OpenAI.

    Args:
        system: Prompt système définissant le rôle
        user: Message utilisateur avec le contexte
        temp: Température (0 = déterministe, >0 = créatif)

    Returns:
        Réponse du modèle
    """
    r = llm.chat.completions.create(
        model="gpt-4o-mini",
        temperature=temp,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user}
        ]
    )
    return r.choices[0].message.content.strip()

@dataclass
class Message:
    """Message échangé entre agents."""
    sender: str
    content: str
    ts: float = field(default_factory=time.time)

@dataclass
class Blackboard:
    """
    Blackboard (tableau noir) : mémoire partagée entre agents.
    - data: dictionnaire de données structurées
    - log: historique des messages
    """
    data: Dict[str, Any] = field(default_factory=dict)
    log: List[Message] = field(default_factory=list)

    def write(self, msg: Message):
        """Ajoute un message au log."""
        self.log.append(msg)

    def update(self, **kwargs):
        """Met à jour les données du blackboard."""
        self.data.update(kwargs)

print("✓ Structures de base définies (Message, Blackboard, fonction chat)")

✓ Structures de base définies (Message, Blackboard, fonction chat)


## 3) Create role prompts :

Définition des prompts pour chaque rôle spécialisé :
- **Researcher** : collecte des faits et sources
- **Planner** : propose un plan structuré
- **Critic** : identifie les lacunes et améliore
- **Single Agent** : combine tous les rôles

In [4]:
ROLE_RESEARCHER = (
    "Role: Researcher. Read the task & propose 3 concise facts or sources "
    "to inform a solution. Output as bullet points. No final answer."
)

ROLE_PLANNER = (
    "Role: Planner. Read the latest facts and propose an ordered plan with 3–5 steps. "
    "State assumptions and risks briefly."
)

ROLE_CRITIC = (
    "Role: Critic. Review the plan for gaps, conflicts, or missing data. "
    "Return 2–3 actionable improvements. No final answer."
)

ROLE_SINGLE = (
    "You are a single agent acting as Researcher, Planner, and Critic at once. "
    "Given the task, produce facts, a plan, then self-critique with improvements."
)

print("✓ Prompts de rôles définis")
print(f"  - Researcher: {len(ROLE_RESEARCHER)} caractères")
print(f"  - Planner: {len(ROLE_PLANNER)} caractères")
print(f"  - Critic: {len(ROLE_CRITIC)} caractères")
print(f"  - Single: {len(ROLE_SINGLE)} caractères")

✓ Prompts de rôles définis
  - Researcher: 132 caractères
  - Planner: 117 caractères
  - Critic: 120 caractères
  - Single: 150 caractères


## 4) Define agent classes :

Classe `BaseAgent` implémentant le comportement de base d'un agent :
- Lecture du contexte (task + blackboard + historique)
- Appel au LLM avec son prompt de rôle
- Retour d'un message structuré

In [5]:
class BaseAgent:
    """
    Agent de base avec un rôle et un nom.
    """
    def __init__(self, name: str, role: str):
        self.name = name
        self.role = role

    def act(self, task: str, bb: Blackboard) -> Message:
        """
        Exécute une action basée sur le task et l'état du blackboard.

        Args:
            task: Tâche à accomplir
            bb: Blackboard contenant l'état partagé

        Returns:
            Message contenant la réponse de l'agent
        """
        # Construction du contexte
        context = (
            f"Task:\n{task}\n\nBlackboard:\n{bb.data}\n\n"
            f"Conversation so far:\n" +
            "\n".join([
                f"- {m.sender}: {m.content[:200]}"
                for m in bb.log[-6:]  # Garde les 6 derniers messages
            ])
        )

        # Appel au LLM
        out = chat(system=self.role, user=context)

        return Message(sender=self.name, content=out)

# Instanciation des agents spécialisés
researcher = BaseAgent("Researcher", ROLE_RESEARCHER)
planner    = BaseAgent("Planner", ROLE_PLANNER)
critic     = BaseAgent("Critic", ROLE_CRITIC)
single     = BaseAgent("Solo", ROLE_SINGLE)

print("✓ Agents instanciés :")
print(f"  - {researcher.name}")
print(f"  - {planner.name}")
print(f"  - {critic.name}")
print(f"  - {single.name} (single-agent)")

✓ Agents instanciés :
  - Researcher
  - Planner
  - Critic
  - Solo (single-agent)


## 5) Turn-based multi-agent loop : ✨ Version améliorée ✨

**Protocole:** Researcher → Planner → Critic → Planner (révision)

### 🆕 Améliorations clés

1. **Awareness multi-rounds** : Le Researcher sait qu'il doit fournir des faits **complémentaires** aux rounds suivants
2. **Contexte enrichi** : Prise en compte de l'historique (4 derniers messages au lieu de 2)
3. **Historique des plans** : Stockage de toutes les versions pour traçabilité
4. **Statistiques détaillées** : Affichage après chaque round
5. **Mode verbose** : Possibilité de désactiver les logs pour benchmark

In [6]:
def summarize_last(msgs: List[Message], role: str) -> str:
    """
    Résume les derniers messages pour un rôle spécifique.
    Utilise le LLM pour condenser l'information.

    Args:
        msgs: Liste des messages
        role: Rôle pour lequel on résume

    Returns:
        Résumé condensé
    """
    if not msgs:
        return ""

    # Prendre plus de contexte pour les rounds suivants (4 au lieu de 2)
    joined = "\n".join([m.content for m in msgs[-4:]])
    return chat(system=f"Summarize key points for {role}.", user=joined)

def multi_agent_run(task: str, rounds: int = 1, verbose: bool = True) -> Blackboard:
    """
    Exécute le workflow multi-agent avec support de multiples rounds.

    Args:
        task: Tâche à accomplir
        rounds: Nombre d'itérations du cycle complet
        verbose: Afficher les logs détaillés

    Returns:
        Blackboard final avec tous les résultats
    """
    bb = Blackboard(data={
        "task": task,
        "facts": [],        # Liste de tous les faits collectés
        "plans": [],        # Historique des plans (pour traçabilité)
        "critiques": [],    # Historique des critiques
        "plan": "",         # Plan actuel (le plus récent)
        "critique": ""      # Critique actuelle
    })

    for r in range(rounds):
        if verbose:
            print(f"\n{'='*70}")
            print(f"🔄 ROUND {r+1}/{rounds}")
            print('='*70)

        # 1) Researcher collecte des faits (avec awareness des rounds précédents)
        if verbose:
            print("  📚 Researcher: collecte des faits...")

        researcher_context = task
        if r > 0:
            # Après le 1er round, demander des faits complémentaires
            researcher_context += (
                f"\n\nPrevious facts collected:\n" +
                "\n".join([f"- {f[:100]}..." for f in bb.data["facts"][-3:]]) +
                "\n\nProvide NEW complementary facts or deeper insights."
            )

        m1 = researcher.act(researcher_context, bb)
        bb.write(m1)
        bb.data["facts"].append(m1.content)

        if verbose:
            print(f"    ✓ {len(m1.content)} caractères collectés")

        # 2) Planner propose un plan
        if verbose:
            print("  📋 Planner: création du plan...")

        ctx = summarize_last(bb.log, "Planner")
        planner_prompt = task + "\n\nContext:\n" + ctx

        if r > 0:
            # Mentionner le plan précédent pour amélioration incrémentale
            planner_prompt += (
                f"\n\nPrevious plan (Round {r}):\n{bb.data['plans'][-1][:200]}..."
                "\n\nRefine and improve based on new facts."
            )

        m2 = planner.act(planner_prompt, bb)
        bb.write(m2)
        bb.data["plan"] = m2.content
        bb.data["plans"].append(m2.content)

        if verbose:
            print(f"    ✓ Plan v{r+1} généré ({len(m2.content)} caractères)")

        # 3) Critic review le plan
        if verbose:
            print("  🔍 Critic: analyse et critique...")

        ctx = summarize_last(bb.log, "Critic")
        m3 = critic.act(task + "\n\nContext:\n" + ctx, bb)
        bb.write(m3)
        bb.data["critique"] = m3.content
        bb.data["critiques"].append(m3.content)

        if verbose:
            print(f"    ✓ Critique générée ({len(m3.content)} caractères)")

        # 4) Planner révise le plan selon la critique
        if verbose:
            print("  ✏️  Planner: révision du plan...")

        ctx = summarize_last(bb.log, "Planner")
        m4 = planner.act(
            task + "\n\nRevise plan per critique:\n" + ctx, bb
        )
        bb.write(m4)
        bb.data["plan"] = m4.content  # Plan révisé devient le plan actuel

        if verbose:
            print(f"    ✓ Plan révisé ({len(m4.content)} caractères)")

        bb.update(round=r+1)

        # Afficher un résumé du round
        if verbose:
            print(f"\n  📊 Round {r+1} terminé:")
            print(f"    - Total messages: {len(bb.log)}")
            print(f"    - Total faits: {len(bb.data['facts'])}")
            print(f"    - Plans générés: {len(bb.data['plans'])}")

    if verbose:
        print("\n" + "="*70)
        print("✅ Multi-agent workflow terminé")
        print("="*70)
        print(f"📈 Statistiques finales:")
        print(f"  - {rounds} round(s) complété(s)")
        print(f"  - {len(bb.log)} messages échangés")
        print(f"  - {len(bb.data['facts'])} faits collectés")
        print(f"  - {len(bb.data['plans'])} versions de plan")

    return bb

print("✓ Fonction multi_agent_run AMÉLIORÉE définie")

✓ Fonction multi_agent_run AMÉLIORÉE définie


## 6) Single-agent baseline :

L'agent unique combine tous les rôles en une seule invocation.

In [7]:
def single_agent_run(task: str) -> Blackboard:
    bb = Blackboard(data={"task": task})
    print("\n🤖 Single-agent: traitement...")
    m = single.act(task, bb)
    bb.write(m)
    bb.update(final=m.content)
    print("✓ Single-agent terminé")
    return bb

print("✓ Fonction single_agent_run définie")

✓ Fonction single_agent_run définie


## 7) Test avec 1 round puis 2 rounds

### Test avec 1 round

In [8]:
TASK = (
    "Design a 3-email outreach sequence for a B2B AI tool targeting "
    "healthcare analytics leaders. Include subject lines and a clear CTA each."
)

print("="*70)
print("TÂCHE À ACCOMPLIR")
print("="*70)
print(TASK)
print("="*70)

TÂCHE À ACCOMPLIR
Design a 3-email outreach sequence for a B2B AI tool targeting healthcare analytics leaders. Include subject lines and a clear CTA each.


In [9]:
# Test avec 1 ROUND
bb_multi_1 = multi_agent_run(TASK, rounds=1)


🔄 ROUND 1/1
  📚 Researcher: collecte des faits...
    ✓ 807 caractères collectés
  📋 Planner: création du plan...
    ✓ Plan v1 généré (2633 caractères)
  🔍 Critic: analyse et critique...
    ✓ Critique générée (979 caractères)
  ✏️  Planner: révision du plan...
    ✓ Plan révisé (2262 caractères)

  📊 Round 1 terminé:
    - Total messages: 4
    - Total faits: 1
    - Plans générés: 1

✅ Multi-agent workflow terminé
📈 Statistiques finales:
  - 1 round(s) complété(s)
  - 4 messages échangés
  - 1 faits collectés
  - 1 versions de plan


### Test avec 2 rounds (pour voir l'amélioration incrémentale)

In [10]:
# Test avec 2 ROUNDS
bb_multi_2 = multi_agent_run(TASK, rounds=2)


🔄 ROUND 1/2
  📚 Researcher: collecte des faits...
    ✓ 751 caractères collectés
  📋 Planner: création du plan...
    ✓ Plan v1 généré (2481 caractères)
  🔍 Critic: analyse et critique...
    ✓ Critique générée (1108 caractères)
  ✏️  Planner: révision du plan...
    ✓ Plan révisé (3846 caractères)

  📊 Round 1 terminé:
    - Total messages: 4
    - Total faits: 1
    - Plans générés: 1

🔄 ROUND 2/2
  📚 Researcher: collecte des faits...
    ✓ 741 caractères collectés
  📋 Planner: création du plan...
    ✓ Plan v2 généré (3917 caractères)
  🔍 Critic: analyse et critique...
    ✓ Critique générée (1128 caractères)
  ✏️  Planner: révision du plan...
    ✓ Plan révisé (4105 caractères)

  📊 Round 2 terminé:
    - Total messages: 8
    - Total faits: 2
    - Plans générés: 2

✅ Multi-agent workflow terminé
📈 Statistiques finales:
  - 2 round(s) complété(s)
  - 8 messages échangés
  - 2 faits collectés
  - 2 versions de plan


In [11]:
# Single-agent pour comparaison
bb_single = single_agent_run(TASK)


🤖 Single-agent: traitement...
✓ Single-agent terminé


## 8) Comparaison des résultats

In [12]:
print("\n" + "="*70)
print("COMPARAISON: 1 ROUND vs 2 ROUNDS vs SINGLE-AGENT")
print("="*70)

print("\n📊 MULTI-AGENT 1 ROUND:")
print(f"  - Messages: {len(bb_multi_1.log)}")
print(f"  - Faits: {len(bb_multi_1.data['facts'])}")
print(f"  - Plans: {len(bb_multi_1.data['plans'])}")

print("\n📊 MULTI-AGENT 2 ROUNDS:")
print(f"  - Messages: {len(bb_multi_2.log)}")
print(f"  - Faits: {len(bb_multi_2.data['facts'])}")
print(f"  - Plans: {len(bb_multi_2.data['plans'])}")

print("\n📊 SINGLE-AGENT:")
print(f"  - Messages: {len(bb_single.log)}")

print("\n" + "="*70)
print("PLAN FINAL (2 rounds):")
print("="*70)
print(bb_multi_2.data.get('plan', ''))


COMPARAISON: 1 ROUND vs 2 ROUNDS vs SINGLE-AGENT

📊 MULTI-AGENT 1 ROUND:
  - Messages: 4
  - Faits: 1
  - Plans: 1

📊 MULTI-AGENT 2 ROUNDS:
  - Messages: 8
  - Faits: 2
  - Plans: 2

📊 SINGLE-AGENT:
  - Messages: 1

PLAN FINAL (2 rounds):
### Ordered Plan for Email Outreach Sequence for B2B AI Tool Targeting Healthcare Analytics Leaders

1. **Email 1: Introduction to AI Benefits in Healthcare**
   - **Subject Line**: Unlock 30% Improvement in Patient Outcomes with AI
   - **Body**: 
     Hi [Recipient's Name],
     
     As a leader in healthcare analytics at [Recipient's Organization], you understand the critical role that data plays in enhancing patient care and operational efficiency. According to McKinsey, organizations leveraging AI for analytics can improve outcomes by up to 30%. Our AI tool is designed to help you harness this potential effectively, especially in addressing challenges like [specific challenge relevant to their organization, e.g., patient data management or oper

In [13]:
# Évolution des faits entre rounds
print("\n" + "="*70)
print("ÉVOLUTION DES FAITS (2 rounds)")
print("="*70)

for i, fact in enumerate(bb_multi_2.data['facts'], 1):
    round_num = (i + 3) // 4  # 4 messages par round
    print(f"\n[Round {round_num}] Fait {i}:")
    print(fact[:300] + "..." if len(fact) > 300 else fact)


ÉVOLUTION DES FAITS (2 rounds)

[Round 1] Fait 1:
- **Fact 1**: According to a report by McKinsey, healthcare organizations that leverage AI for analytics can improve patient outcomes and operational efficiency by up to 30%. This highlights the potential value of AI tools in healthcare analytics.

- **Fact 2**: A survey by Gartner indicates that 75...

[Round 1] Fait 2:
- **Fact 4**: A study published in the Journal of Medical Internet Research found that AI-driven analytics can reduce hospital readmission rates by up to 20%, demonstrating the tool's potential to enhance patient care and reduce costs in healthcare settings.

- **Fact 5**: According to a report by A...


## Conclusion

### 🎯 Ce que vous avez appris

1. **Multi-rounds fonctionnent** : Chaque round améliore le précédent
2. **Awareness contextuelle** : Les agents savent où ils en sont dans le processus
3. **Incrémentalité** : Le Researcher apporte de nouveaux faits à chaque round
4. **Trade-off coût/qualité** : 2 rounds = 2x plus d'appels LLM mais meilleure qualité

### 🔬 Expérimentations suggérées

- Tester avec 3-4 rounds : à quel moment observe-t-on des rendements décroissants ?
- Ajouter un critère de convergence : arrêter si la critique devient positive
- Comparer les coûts : mesurer tokens utilisés par round
- Tester sur des tâches plus complexes nécessitant vraiment plusieurs itérations